In [1]:
import numpy as np 
import pandas as pd 
import altair as alt 
import os 
import re 
import ntpath
from Commons.data_processing import *
from Commons.DataProcessors.pd_processor import PDProcessor

In [2]:
files = get_files(r".\N_Glycosylation_Results")

In [3]:
read_data = None

for file in files:

    base_name, _ = ntpath.splitext(ntpath.basename(file))
    div = base_name.split("-")[0]
    div = div.split("_")
    conc, temp, run = div[-3:]

    data = PDProcessor([file], sample_name="_".join(div[-3:]))
    data.add_special_column("concentration", conc)
    data.add_special_column("temperature", temp)
    data.add_special_column("run", run)

    if read_data is None:
        read_data = data
    else:
        read_data.join_processors(data)


read_data.alias_engine("")


KeyboardInterrupt: 

In [ ]:
peptides = read_data.peptides[~read_data.peptides.glycan_composition.isna()]
conc_map = {"1": 1.0, "05": 0.5, "025": 0.25, "0125": 0.125, "00625": 0.0625}
peptides.loc[:, 'concentration'] = peptides.concentration.map(conc_map)


C:\Users\Graham Delafield\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-4-2ccea1480c75>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peptides.loc[:, 'concentration'] = peptides.concentration.map(conc_map)


In [ ]:
cutoffs = [50, 100, 150, 200, 250, 300, 350, 400, 500]
nums = []
for cut in cutoffs:
    nums.append(len(peptides[peptides.byonic_score >= cut]))

res = pd.DataFrame({"cutoffs": cutoffs, "vals": nums})

alt.Chart(res).mark_bar().encode(x="cutoffs:N", y="vals:Q")

alt.Chart(...)

In [ ]:
def categorize_glycan(glycan):

    glycan = glycan.replace(")", ",")
    glycan = glycan.replace("(", " ")
    glycan = glycan.split(",")[:-1]
    d = {k: int(v) for [k, v] in [i.split(" ") for i in glycan]}

    if "NeuAc" in d or "NeuGc" in d:
        return "Sialylated"
    elif "Fuc" in d:
        if d["HexNAc"] > 2:
            return "Fucosylated"
        elif d["HexNAc"] == 2:
            if "Hex" in d:
                if d["Hex"] > 4:
                    return "Complex"
            else:
                return "Paucimannose"
    elif d["HexNAc"] > 2:
        return "Complex"
    elif d["HexNAc"] <= 2:
        if "Hex" in d:
            if d["Hex"] <= 9 and d["Hex"] > 4:
                return "High Mannose"

        return "Paucimannose"

def determine_degree_sial(glycan):

    degree_map = {
        1: 'Monosialylated',
        2: 'Disialylated',
        3: 'Trisialylated',
        4: 'Tetrasialylated',
        5: 'Pentasialylated',
    }

    glycan = glycan.replace(')', ',')
    glycan = glycan.replace('(', ' ')
    glycan = glycan.split(',')[:-1]
    d = {k:int(v) for [k, v] in [i.split(' ') for i in glycan]}

    if not 'NeuAc' in d and 'NeuGc' not in d:
        return 0
    else:
        total = int(d.get('NeuAc', 0)) + int(d.get('NeuGc', 0))
        return degree_map[total]

peptides = peptides[peptides.byonic_score >= 300]
peptides['glycan'] = peptides.glycan_composition

peptides.loc[:, "glycan_type"] = peptides.glycan.map(categorize_glycan)
peptides.loc[:, "degree_sial"] = peptides.glycan.map(determine_degree_sial)

# map unique glycopeptide id

peptides.loc[:, "pep_mods"] = peptides.apply(
    lambda x: x["sequence"] + "_" + x["modifications"], axis=1
)
peptides = peptides.reset_index()
peptides = peptides.sort_values('concentration', ascending=False)
peptides.head(3)

,index,accession,description,checked,confidence,annotated_sequence,modifications,num_protein_groups,num_proteins,num_psms,...,positions_in_proteins,sequence,data_source,concentration,temperature,run,glycan,glycan_type,degree_sial,pep_mods
2475,423,P61823,Ribonuclease pancreatic OS=Bos taurus OX=9913 ...,True,High,[K].SRNLTK.[D],1xHexNAc(2)Hex(8) [N3],1,1,7,...,[58-63],SRNLTK,1_60C_Run3,1,60C,Run3,HexNAc(2)Hex(8),High Mannose,0,SRNLTK_1xHexNAc(2)Hex(8) [N3]
2007,277,P61823,Ribonuclease pancreatic OS=Bos taurus OX=9913 ...,True,High,[R].NLTKDR.[C],1xHexNAc(1) [N1],1,1,2,...,[60-65],NLTKDR,1_30C_Run3,1,30C,Run3,HexNAc(1),Paucimannose,0,NLTKDR_1xHexNAc(1) [N1]
2015,306,P61823,Ribonuclease pancreatic OS=Bos taurus OX=9913 ...,True,Low,[R].QHMDSSTSAASSSNYCNQMMKSRNLTK.[D],1xCarbamidomethyl [C16]; 1xDeamidated [Q18]; 1...,1,1,1,...,[37-63],QHMDSSTSAASSSNYCNQMMKSRNLTK,1_30C_Run3,1,30C,Run3,HexNAc(5)Hex(4),Complex,0,QHMDSSTSAASSSNYCNQMMKSRNLTK_1xCarbamidomethyl ...


In [ ]:
def iterate_contents(column_name, dataframe, get_item=False):
    for unique_item in dataframe[column_name].unique():
        result = dataframe[dataframe[column_name]==unique_item]
        if get_item:
            yield unique_item, result
        else:
            yield result

# find peptides that are found in at least 2 runs at each concentration
multiple_hits = pd.DataFrame()
for temperature in iterate_contents('temperature', peptides):
    for concentration in iterate_contents('concentration', temperature):
        counts = concentration.pep_mods.value_counts()
        valid = counts[counts.values >= 2].keys()
        to_keep = concentration[concentration.pep_mods.isin(valid)]
        multiple_hits = pd.concat([multiple_hits, to_keep])

In [77]:
multiple_hits.shape

(2255, 42)

# What are we even doing?

`Intact Glycopeptide Analysis at Elevated Temperatures Reveals limitations and Considerations in Label Free Quantitation`

1. We know that different temperatures enable different levels of performance in glycopeptide identification
- Show the difference between temperatures (num glycopeptideP)
- Pinopint what is responsible. Increased peak area? Peak intensity? Is the separation of isomers somehow causing identification errors?

2. Is this a global occurrence or are various glycan types affected differently?
- For each temperature and glycan types, show the relative identification rates
- Using the pinpointed cause of different performance from aim1, show positive and negative examples for glycan types.

3. How does this effect quantitation?
- Is LFQ even remotely possible when temperature is varied?
- Is the ability to do LFQ rescued when analyzing at one temperature?
- Can we achieve greater LOQ when running at the optimal temperature?
- Are there any glycan types that show no ability to do LFQ?
- Is there a method of LFQ that is more useful than the other?

## 1a

Taking the max concentration, what is the difference between temperature?

In [144]:
# information to extract: num peptides, num unique glycans, num unique peptide backbones
temps, vals, kind, concent = [], [], [], []

for conc, conc_frame in iterate_contents('concentration', multiple_hits, get_item=True):
    for temp, temp_frame in iterate_contents('temperature', conc_frame, get_item=True):
        for run, run_frame in iterate_contents('run', temp_frame, get_item=True):

            unique_gly = len(run_frame.pep_mods.unique())
            concent.append(conc)
            vals.append(unique_gly)
            temps.append(temp)
            kind.append("Unique Glycopeptides")

            unique_seq = len(run_frame.sequence.unique())
            concent.append(conc)
            vals.append(unique_seq)
            temps.append(temp)
            kind.append("Unique Backbones")

            unique_glycans = len(run_frame.glycan.unique())
            concent.append(conc)
            vals.append(unique_glycans)
            temps.append(temp)
            kind.append("Unique Glycans")

res = pd.DataFrame({"Temperature": temps, "Kind": kind, "Values": vals, "Concentration": concent})

my_colors = alt.Color(
    "Temperature:N",
    scale=alt.Scale(
        domain=["30C", "45C", "60C"], range=["#6E6581", "#B0B2BB", "#6B8A97"]
    ),
)

conc_max_comparison = (
    alt.Chart(res)
    .mark_bar()
    .encode(
        x=alt.X("Temperature:N", axis=alt.Axis(labelAngle=-45)),
        y=alt.Y("mean(Values):Q", title="Number of Identifications"),
        color=my_colors
    )
).properties(
    width=75,
    height=75
)

cmc_err_bars = (
    alt.Chart(res)
    .mark_errorbar(extent='stdev')
    .encode(
        x=alt.X("Temperature:N", title="", axis=alt.Axis(labelAngle=-45)),
        y=alt.Y("mean(Values):Q", title=""),
    )
)

base = alt.layer(conc_max_comparison, cmc_err_bars).facet(
    column="Kind:N", spacing=50,
)

max_conc = base.transform_filter(
        alt.datum.Concentration == 1.0
    ).properties(
        title='Temperature Differences, Max Concentration'
    )


all_conc = alt.vconcat()
for concentration in res.Concentration.unique():
    all_conc &= base.transform_filter(alt.datum.Concentration==concentration).properties(title=f'Concentration = {concentration}')

all_conc

alt.VConcatChart(...)

# Observations
When comparing across temperatures, there is often a slight difference in number of identifications between 30C and 45C. However, this difference is small and shows no consistent trend between concentrations analyzed.

However, at all concentrations tested, their is a significant difference when running at 60C. What could be causing this?



## 1b

What is a main source of this difference?

## 2a

Do all glycan types experience this trend?

In [98]:
glycan_sub = multiple_hits[
    [
        "sequence",
        "glycan",
        "glycan_type",
        "pep_mods",
        "temperature",
        "concentration",
        "run",
    ]
]

glycan_color_scale = alt.Scale(
    domain=[
        'Sialylated',
        'Fucosylated',
        'Complex',
        'Paucimannose',
        'High Mannose',
    ],
    range=[
        '#6E6581',
        '#AF3A53',
        '#6B8A97',
        '#B0B2BB',
        '#658A64',
    ]
)

bars = (
    alt.Chart(glycan_sub)
    .mark_bar()
    .encode(
        x=alt.X("temperature:N", title="", axis=alt.Axis(labelAngle=-45)),
        y=alt.Y("mean(my_count):Q", title="Number Glycan Matches"),
        color=alt.Color("glycan_type:N", scale=glycan_color_scale),
    )
    .transform_aggregate(
        my_count='count(glycan_type)',
        groupby=['glycan_type', 'concentration', 'temperature', 'run']
    )
)

err = (
    alt.Chart(glycan_sub)
    .mark_errorbar(extent='stdev')
    .encode(
        x=alt.X("temperature:N", title="", axis=alt.Axis(labelAngle=-45)),
        y=alt.Y("mean(my_count):Q", title="Number Glycan Matches"),
    )
    .transform_aggregate(
        my_count='count(glycan_type)',
        groupby=['glycan_type', 'concentration', 'temperature', 'run']
    )
)

glyc_by_concentration = alt.layer(bars, err).facet(
    column=alt.Column('glycan_type', title=''),
    row='concentration:N'
)

conc_bars = alt.vconcat()
for concentration in glycan_sub.concentration.unique():
    conc_bars &= base.transform_filter(alt.datum.concentration == concentration).properties(
        title=f"Concentration = {concentration}"
    )
conc_bars.configure_concat(
    spacing=50
)

glyc_by_concentration

alt.FacetChart(...)

At all concentrations, complex and sialylated glycopeptides show decreasing numbers of positive identifications as temperature increases. Fucosylated and High mannose glycopeptides follow this trend at greater dilutions but resist at higher analyte concentrations. In general, we see an increase in the number of paucimannose identifications -- could we be promoting premature glycan dissociation at higher temperatures? However, the number of paucimannose glycopepitdes gained (<5) as temperature increases does not match the number of glycopeptides lost in other categories, so it is unlikely the sole cause of decreased identifications is premature dissociation.

What could be other causes? 

- Reduced peak area causing them to not be selected? (check for mass in ms1 but not in ms2)

- Poor fragmentation efficiency? (Compare MS2 from positive and negative runs) I evaluated this briefly in the ABC manuscript, temp didn't seem to effect it.

- ## <b> Could we be seeing a change in number of sialic acids as temp changes?? </b>

## 2b

Let's investigate if the number of PSMs or peak area could show us why this is happening.

In [83]:
psms = read_data.psms[(read_data.psms.byonic_score >= 300) & (~read_data.psms.glycan_composition.isna())]
psms.loc[:, 'concentration'] = psms.concentration.map(conc_map)
psms.loc[:, 'glycan'] = psms.glycan_composition.map(categorize_glycan)
psms = psms.sort_values('concentration', ascending=False)

psms.loc[:, "pep_mods"] = psms.apply(
    lambda x: x["sequence"] + "_" + x["modifications"], axis=1
)

psms = psms.reset_index()

C:\Users\Graham Delafield\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
C:\Users\Graham Delafield\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Graham Delafield\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.

In [84]:
spec_count = psms[
    ["sequence", "glycan", "temperature", "concentration", "run", "pep_mods"]
]

boxes = (
    alt.Chart(spec_count)
    .mark_boxplot()
    .encode(
        x=alt.X("temperature:N",
                axis=alt.Axis(labelAngle=-45)),
        y="my_count:Q",
        color=alt.Color("glycan:N", scale=glycan_color_scale),
    )
    .transform_aggregate(
        my_count="count(pep_mods)",
        groupby=["temperature", "run", "concentration", "glycan"],
    )
)

conc_boxes = alt.vconcat()
for concentration in spec_count.concentration.unique():
    conc_boxes &= boxes.transform_filter(
        alt.datum.concentration == concentration
    ).properties(title=f"Concentration = {concentration}")

test = alt.layer(boxes).facet(
    column=alt.Column('glycan:N', title=''),
    row=alt.Row('concentration:N', title='')
)

In [85]:
glyc_by_concentration | test

alt.HConcatChart(...)

In [104]:
test = multiple_hits[
    [
        "sequence",
        "glycan",
        "glycan_type",
        "pep_mods",
        "temperature",
        "concentration",
        "run",
        "degree_sial",
    ]
]

test = test[test.degree_sial != 0]


sial = alt.Chart(test).mark_bar().encode(
    x=alt.X(
        "temperature:N",
        title="",
        axis=alt.Axis(labelAngle=-45)
    ),
    y=alt.Y("mean(my_count):Q"),
    color='degree_sial:N'
).transform_aggregate(
    my_count = 'count(degree_sial):Q',
    groupby = ['temperature', 'run', 'concentration', 'degree_sial']
)

err = alt.Chart(test).mark_errorbar(extent='stdev').encode(
    x=alt.X(
        "temperature:N",
        title="",
        axis=alt.Axis(labelAngle=-45)
    ),
    y=alt.Y("mean(my_count):Q"),
).transform_aggregate(
    my_count = 'count(degree_sial):Q',
    groupby = ['temperature', 'run', 'concentration', 'degree_sial']
)

alt.layer(sial, err).facet(
    column=alt.Column('degree_sial:O'),
    row=alt.Row('concentration:Q')
)


alt.FacetChart(...)

## Possible Observation

When looking at glycan types across all concentrations and temperatures, the only types that resist the trend of decreasing in accordance with temperature is paucimannose and sialylated. However, if we consider the degree of sialylation, this does follow the trend of decreasing as temperature increases. 

How can we verify or check to see if this is indeed the case.

### Look for instances where a glycopeptide is identified at 30C and not 45C. Then see if at the same retention time we see a new/different glycan attached to the same backbone.

In [142]:
test_conc = test[test.concentration==1.0]
test_30 = test_conc[test_conc.temperature=="30C"]
test_45 = test_conc[test_conc.temperature=="45C"]

x = set(test_30.pep_mods.tolist()) - set(test_45.pep_mods.tolist())
y = set(test_45.pep_mods.tolist()) - set(test_30.pep_mods.tolist())

In [143]:
x

{'CVYNCSFIK_2xCarbamidomethyl [C1; C5]; 1xHexNAc(4)Hex(5)NeuAc(1)NeuGc(1) [N4]',
 'CVYNCSFIK_2xCarbamidomethyl [C1; C5]; 1xHexNAc(4)Hex(5)NeuAc(2) [N4]',
 'LCPDCPLLAPLNDSR_2xCarbamidomethyl [C2; C5]; 1xHexNAc(4)Hex(5)NeuGc(2) [N12]',
 'LCPDCPLLAPLNDSR_2xCarbamidomethyl [C2; C5]; 1xHexNAc(5)Hex(6)NeuAc(3) [N12]',
 'NLTK_1xHexNAc(4)Hex(6)NeuAc(1) [N1]',
 'NPEYNK_1xHexNAc(5)Hex(6)NeuAc(2) [N5]',
 'QNGTLSK_1xHexNAc(4)Hex(5)NeuAc(3) [N2]'}